In [13]:
from bs4 import BeautifulSoup as bs

import requests
import re

import pyodbc

import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.sql import text

import pandas as pd
import numpy as np
import pyodbc

from datetime import datetime
import time
import random

import os

In [15]:
schedule_url = 'https://www.cbssports.com/nfl/news/2024-nfl-team-by-team-schedules-times-dates-tv-for-the-entire-season-for-all-32-franchises/'
schedule_url

'https://www.cbssports.com/nfl/news/2024-nfl-team-by-team-schedules-times-dates-tv-for-the-entire-season-for-all-32-franchises/'

In [17]:
#get team table
teams_schedule_2024_li = pd.read_html(schedule_url, attrs={'class':'TableBuilder'})

In [19]:
teams_schedule_2024_li

[          Date                                      Matchup Time (ET)     TV
 0     9/8/2024           Arizona Cardinals at Buffalo Bills     1:00p    CBS
 1    9/15/2024       Arizona Cardinals vs. Los Angeles Rams     4:05p    FOX
 2    9/22/2024          Arizona Cardinals vs. Detroit Lions     4:25p    FOX
 3    9/29/2024  Arizona Cardinals vs. Washington Commanders     4:05p    FOX
 4    10/6/2024     Arizona Cardinals at San Francisco 49ers     4:05p    FOX
 5   10/13/2024       Arizona Cardinals at Green Bay Packers     1:00p    FOX
 6   10/21/2024   Arizona Cardinals vs. Los Angeles Chargers     9:00p  ESPN+
 7   10/27/2024          Arizona Cardinals at Miami Dolphins     1:00p    FOX
 8    11/3/2024          Arizona Cardinals vs. Chicago Bears     4:05p    CBS
 9   11/10/2024          Arizona Cardinals vs. New York Jets     4:25p    CBS
 10         NaN                                  Week 11 bye       NaN    NaN
 11  11/24/2024        Arizona Cardinals at Seattle Seahawks    

In [21]:
# delay execute so not to trigger ip address block
time.sleep(random.randrange(7,9))

In [22]:
team_li = [
    'Cardinals',
    'Falcons',
    'Ravens',
    'Bills',
    'Panthers',
    'Bears',
    'Bengals',
    'Browns',
    'Cowboys',
    'Broncos',
    'Lions',
    'Packers',
    'Texans',
    'Colts',
    'Jaguars',
    'Chiefs',
    'Raiders',
    'Chargers',
    'Rams',
    'Dolphins',
    'Vikings',
    'Patriots',
    'Saints',
    'Giants',
    'Jets',
    'Eagles',
    'Steelers',
    '49ers',
    'Seahawks',
    'Buccaneers',
    'Titans',
    'Commanders'
]

In [23]:
year = 2024

for df, team in zip(teams_schedule_2024_li, team_li):
    #add year column
    df['Year'] = [year] * len(df)

   #add team column
    df['Team'] = [team] * len(df)

    #opponent name
    opp_dict = {'Opponent': []}
    for i in range(0, len(df)):
        opp = df['Matchup'].iloc[i].split()[-1]

        opp_dict['Opponent'].append(opp)
        
    df['Opponent'] = opp_dict['Opponent']

    df.to_csv(f'NFL_Data_2024/{team}/Schedule')

In [24]:
pd.read_csv('NFL_Data_2024/Rams/Schedule', index_col=0)

,Date,Matchup,Time (ET),TV,Year,Team,Opponent
0,9/8/2024,Los Angeles Rams at Detroit Lions,8:20p,NBC,2024,Rams,Lions
1,9/15/2024,Los Angeles Rams at Arizona Cardinals,4:05p,FOX,2024,Rams,Cardinals
2,9/22/2024,Los Angeles Rams vs. San Francisco 49ers,4:25p,FOX,2024,Rams,49ers
3,9/29/2024,Los Angeles Rams at Chicago Bears,1:00p,FOX,2024,Rams,Bears
4,10/6/2024,Los Angeles Rams vs. Green Bay Packers,4:25p,CBS,2024,Rams,Packers
5,NaN,Week 6 bye,NaN,NaN,2024,Rams,bye
6,10/20/2024,Los Angeles Rams vs. Las Vegas Raiders,4:05p,CBS,2024,Rams,Raiders
7,10/24/2024,Los Angeles Rams vs. Minnesota Vikings,8:15p,Prime Video,2024,Rams,Vikings
8,11/3/2024,Los Angeles Rams at Seattle Seahawks,4:25p,FOX,2024,Rams,Seahawks
9,11/11/2024,Los Angeles Rams vs. Miami Dolphins,8:15p,ESPN,2024,Rams,Dolphins


#connect schedule to sql
# union of all season and team to one table for each table
servername = 'DESKTOP-5IAPFQC'
football_data = 'Football_Data'
trusted_conneciton = '?trusted_conneciton=yes'
driver = '&driver=ODBC+Driver+17+for+SQL+Server'
username = 'MAKeith92'
password = 'Lopez!123'

#create engine url to connect to sql server
engine_football_data = create_engine(f'mssql+pyodbc://@{servername}/{football_data}{trusted_conneciton}{driver}')
engine_football_data

conn_engine_football_data = engine_football_data.connect()

In [36]:
teams_schedule_df = pd.DataFrame()

for team in team_li:

    df = pd.read_csv(f'NFL_Data_2024/{team}/Schedule', index_col=0)

    teams_schedule_df = pd.concat([teams_schedule_df, df], axis=0)

In [42]:
teams_schedule_df.reset_index(inplace=True, drop=True)
teams_schedule_df

,Date,Matchup,Time (ET),TV,Year,Team,Opponent
0,9/8/2024,Arizona Cardinals at Buffalo Bills,1:00p,CBS,2024,Cardinals,Bills
1,9/15/2024,Arizona Cardinals vs. Los Angeles Rams,4:05p,FOX,2024,Cardinals,Rams
2,9/22/2024,Arizona Cardinals vs. Detroit Lions,4:25p,FOX,2024,Cardinals,Lions
3,9/29/2024,Arizona Cardinals vs. Washington Commanders,4:05p,FOX,2024,Cardinals,Commanders
4,10/6/2024,Arizona Cardinals at San Francisco 49ers,4:05p,FOX,2024,Cardinals,49ers
...,...,...,...,...,...,...,...
571,NaN,Week 14 bye,NaN,NaN,2024,Commanders,bye
572,12/15/2024,Washington Commanders at New Orleans Saints,1:00p,FOX,2024,Commanders,Saints
573,12/22/2024,Washington Commanders vs. Philadelphia Eagles,1:00p,FOX,2024,Commanders,Eagles
574,12/28/2024,Washington Commanders vs. Atlanta Falcons,TBD,TBD,2024,Commanders,Falcons


In [43]:
teams_schedule_df.to_sql('2024_Schedule', con=conn_engine_football_data, if_exists='replace')

52